In [22]:
from itertools import chain, combinations

In [9]:
class Graph:
    def __init__(self,edges):
        self.edges = edges
        self.nodes = sorted({ f for f,t in edges }.union({ t for f,t in edges }))

In [131]:
# Graph from the DPccp paper

g1 = Graph({("R0","R1"),
            ("R0","R2"),
            ("R0","R3"),
            ("R1","R4"),
            ("R2","R3"),
            ("R2","R4"),
            ("R3","R4")})

In [11]:
dptable = {}

In [50]:
def makeBi(G,n):
    return {x for x in G.nodes if x <= n}

In [51]:
def Neighborhood(G,S):
    res = []
    for s in S:
        res += [ x for x,y in G.edges if y==s]
        res += [ y for x,y in G.edges if x==s]
    return set(res)

In [52]:
# Enumerate subsets with subsets first
def enumerateSubsets(S):
    sS = sorted(S)
    "powerset([1,2,3]) → (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    return chain.from_iterable(combinations(sS, r) for r in range(1,len(sS)+1))

In [53]:
def enumerateCsg(G):
    for n in reversed(G.nodes):
        yield {n}
        for x in enumerateCsgRec(G,{n},makeBi(G,n)):
            yield x

In [146]:
def enumerateCsgRec(G,S,X):
    # print("calling enumerateCsgRec with S=",S, ", X=", X)
    N = Neighborhood(G,S).difference(X)
    for S1 in enumerateSubsets(N):
        yield S.union(set(S1))
    for S1 in enumerateSubsets(N):
        for x in enumerateCsgRec(G,S.union(set(S1)),X.union(N)):
            yield x

In [106]:
def makeB(G,N,i):
    return {x for x in N if x <= i }

In [145]:
def enumerateCmp(G,S):
    X = makeBi(G,min(S)).union(S)
    N = Neighborhood(G,S).difference(X)
    for n in reversed(sorted(N)):
        yield {n}
        for x in enumerateCsgRec(G,{n},X.union(makeB(G,N,n))):
            yield x

In [164]:
def opt(G):
    # Create DP table:
    dpTable = {}
    counter = 0
    
    # Create duplicate pair table for correctness testing
    dupTable = {}
    
    # iterate over enumeration of connected subgraphs:
    for x in enumerateCsg(G):
        # insert the subgraph into dptable if its a singleton
        fx = frozenset(x)
        if len(x)==1:
            dpTable[fx] = True
            print ("Added CSG: ",x)
            counter += 1
        
        # Now compute all the complements of the subset
        for y in enumerateCmp(G,x):
            # check that the compliment is already in the dpTable
            fy = frozenset(y)
            if fy not in dpTable:
                print("ERROR (cmp): key ", sorted(y), " is not in the table, but should be")

            print ("Added PAIR: ", sorted(x), " and ", sorted(y))
            if frozenset({fx,fy}) in dupTable:
                print("ERROR: pair (", sorted(x), ",", sorted(y), ") is in the table, but should not be")
                
            dupTable[ frozenset({fx,fy}) ] = True


            # add the combintation of csg and cmp to the dpTable
            combo = x.union(y)
            comboKey = frozenset(combo)
            dpTable[comboKey] = True
            counter += 1
            
    print ("Total pairs considered:",counter)

In [162]:
print("\n".join([repr(sorted(x)) for x in list(enumerateCsg(g1))]))

['R4']
['R3']
['R3', 'R4']
['R2']
['R2', 'R3']
['R2', 'R4']
['R2', 'R3', 'R4']
['R1']
['R1', 'R4']
['R1', 'R2', 'R4']
['R1', 'R3', 'R4']
['R1', 'R2', 'R3', 'R4']
['R0']
['R0', 'R1']
['R0', 'R2']
['R0', 'R3']
['R0', 'R1', 'R2']
['R0', 'R1', 'R3']
['R0', 'R2', 'R3']
['R0', 'R1', 'R2', 'R3']
['R0', 'R1', 'R4']
['R0', 'R2', 'R4']
['R0', 'R3', 'R4']
['R0', 'R1', 'R2', 'R4']
['R0', 'R1', 'R3', 'R4']
['R0', 'R2', 'R3', 'R4']
['R0', 'R1', 'R2', 'R3', 'R4']


In [159]:
print(list(enumerateCmp(g1,{'R2'})))

[{'R4'}, {'R3'}, {'R4', 'R3'}]


In [165]:
opt(g1)

Added CSG:  {'R4'}
Added CSG:  {'R3'}
Added PAIR:  ['R3']  and  ['R4']
Added CSG:  {'R2'}
Added PAIR:  ['R2']  and  ['R4']
Added PAIR:  ['R2']  and  ['R3']
Added PAIR:  ['R2']  and  ['R3', 'R4']
Added PAIR:  ['R2', 'R3']  and  ['R4']
Added PAIR:  ['R2', 'R4']  and  ['R3']
Added CSG:  {'R1'}
Added PAIR:  ['R1']  and  ['R4']
Added PAIR:  ['R1']  and  ['R2', 'R4']
Added PAIR:  ['R1']  and  ['R3', 'R4']
Added PAIR:  ['R1']  and  ['R2', 'R3', 'R4']
Added PAIR:  ['R1', 'R4']  and  ['R3']
Added PAIR:  ['R1', 'R4']  and  ['R2']
Added PAIR:  ['R1', 'R4']  and  ['R2', 'R3']
Added PAIR:  ['R1', 'R2', 'R4']  and  ['R3']
Added PAIR:  ['R1', 'R3', 'R4']  and  ['R2']
Added CSG:  {'R0'}
Added PAIR:  ['R0']  and  ['R3']
Added PAIR:  ['R0']  and  ['R3', 'R4']
Added PAIR:  ['R0']  and  ['R2']
Added PAIR:  ['R0']  and  ['R2', 'R3']
Added PAIR:  ['R0']  and  ['R2', 'R4']
Added PAIR:  ['R0']  and  ['R2', 'R3', 'R4']
Added PAIR:  ['R0']  and  ['R1']
Added PAIR:  ['R0']  and  ['R1', 'R4']
Added PAIR:  ['R0'] 